# Accessing SSA Model Data

API version: 1.0

Authors: Craig Pellegrino (NASA GSFC)

## Introduction

This notebook will demonstrate how to access Science Situational Awareness (SSA) model data from the ACROSS core-server using the client library. SSA model data includes `Observatory`/`Telescope`/`Instrument` objects, `Filters`, and `Observations` and `Schedules`. Each of these objects has an API class accessed through the `Client` that can be used to perform `GET` requests to the core-server, retrieving and filtering data based on the user's input parameters.

## 1. Setup and Instantiating the `Client`

The first step to accessing the core-server is to instantiate the `Client` class. Users can pass credentials to the `Client` to perform access-limited tasks, such as `POSTing` schedule data to the server. However all SSA-related `GET` endpoints are available without authentication, so for our purposes here we won't have to worry about credentials.

Let's import the necessary dependencies and instantiate the `Client` object:

In [1]:
from across.client import Client

client = Client()

That's it! `client` can now access the SSA API endpoints to retrieve data. Let's show a few examples of how to do that, including filtering by some common parameters.

## 2. Accessing Observatory/Telescope/Instrument/Filter Data

### 2.1. Observatories

In the ACROSS system, observational resources are captured as `Observatory`, `Telescope`, and `Instrument` objects. The hierarchy is in that order--an `Observatory` can have many `Telescopes`, which in turn can have many `Instruments`. `Instruments` can also have many `Filters`, which describe the wavelength ranges that `Instrument` can observe in a given exposure.

At the top level, an `Observatory` contains data such as a `name`, `type` (`SPACE_BASED` or `GROUND_BASED`), and a collection of telescopes. We can filter and search on any one of those parameters using the client's `observatory.get_many` method, which returns a list of `Observatory` objects that match the input parameters:

In [2]:
# GET an observatory by name
observatories = client.observatory.get_many(name="JWST")
observatory = observatories[0]
observatory.name

'James Webb Space Telescope'

In [3]:
# GET a list of observatories by type
space_based_observatories = client.observatory.get_many(type="SPACE_BASED")
for obs in space_based_observatories:
    print(obs.name)

Imaging X-Ray Polarimetry Explorer
Fermi Gamma-ray Space Telescope
Transiting Exoplanet Survey Satellite
Nuclear Spectroscopic Telescope Array
Neutron Star Interior Composition Explorer
Chandra X-ray Observatory
Neil Gehrels Swift Observatory
Hubble Space Telescope
James Webb Space Telescope
X-ray Multi-Mirror Mission


In [4]:
# GET an observatory by its telescope name
observatories = client.observatory.get_many(telescope_name="UVOT")
observatories[0].name

'Neil Gehrels Swift Observatory'

We can also use the `get` method to look up an observatory by its known `UUID`:

In [5]:
jwst = client.observatory.get(id=observatory.id)
jwst.name

'James Webb Space Telescope'

### 2.2. Telescopes

`Telescopes` function similarly to `Observatories` in that they can be filtered by `name`, `id`, or `Instrument` parameters using the `get_many` method:

In [6]:
# GET a telescope by name
telescopes = client.telescope.get_many(name="UVOT")
telescope = telescopes[0]
telescope.name

'UV/Optical Telescope'

In [7]:
# GET a telescope by its instrument name
telescopes = client.telescope.get_many(instrument_name="NIRCam")
telescopes[0].name

'James Webb Space Telescope'

The `get_many` method also has two optional flags: `include_filters` and `include_footprints`. By default, the `Filters` and `Footprints` for the `Instruments` belonging to the retrieved `Telescope` objects are not returned. However, setting either of these flags to `True` will return their associated data.

`Filters` describe the wavelength ranges an `Instrument` can observe. For more detail, see Section 2.4 below. Here is a quick example demonstrating accessing `Filter` information for an `Instrument` on the retrieved `Telescope`:

In [8]:
# Return filters for the instruments on a telescope
telescopes = client.telescope.get_many(name="XRT", include_filters=True)
filt = telescopes[0].instruments[0].filters[0]
print(f"Name: {filt.name}")
print(f"Min wavelength: {filt.min_wavelength:.1f} Angstrom")
print(f"Max wavelength: {filt.max_wavelength:.1f} Angstrom")

Name: Swift XRT
Min wavelength: 1.2 Angstrom
Max wavelength: 41.3 Angstrom


`Footprints` describe the area of the sky that a detector covers in a single observation. In the ACROSS system, they are represented by a series of points defining the vertices of a polygon, in units of degrees:

In [9]:
# Return footprints for the instruments on a telescope
telescopes = client.telescope.get_many(name="UVOT", include_footprints=True)
telescopes[0].instruments[0].footprints[0]

[Point(x=-0.1416666666666667, y=-0.1416666666666667),
 Point(x=0.1416666666666667, y=-0.1416666666666667),
 Point(x=0.1416666666666667, y=0.1416666666666667),
 Point(x=-0.1416666666666667, y=0.1416666666666667),
 Point(x=-0.1416666666666667, y=-0.1416666666666667)]

Finally, there's a `get` method to look up a `Telescope` by `UUID`:

In [10]:
uvot = client.telescope.get(id=telescope.id)
uvot.name

'UV/Optical Telescope'

### 2.3. Instruments

Finally, `Instrument` objects have analogous filtering parameters to `Observatory` and `Telescope` objects with similar `get` and `get_many` methods:

In [11]:
# GET an instrument by name
instruments = client.instrument.get_many(name="NIRCAM")
instrument = instruments[0]
instrument.name

'Near-Infrared Camera'

In [12]:
# GET an instrument by id
nircam = client.instrument.get(id=instrument.id)
nircam.name

'Near-Infrared Camera'

### 2.4. Filters

`Instruments` can have many `Filters`, which describes the wavelength ranges an `Instrument` can cover in a single observation.

In [13]:
# GET filters for an instrument
filters = client.filter.get_many(instrument_name="UVOT")
filt = filters[0]
print(f"Name: {filt.name}")
print(f"Min wavelength: {filt.min_wavelength:.1f}")
print(f"Max wavelength: {filt.max_wavelength:.1f}")

Name: Swift UVOT UVW2
Min wavelength: 1600.0
Max wavelength: 2250.0


Suppose you want to observe over some wavelength range using a given instrument. You can use the client to find all the filters that match these criteria: 

In [14]:
# GET filters by wavelength region
filters = client.filter.get_many(instrument_name="UVOT", covers_wavelength=2300)  # Assumes Angstroms
print([filt.name for filt in filters])

['Swift UVOT UVM2', 'Swift UVOT UVW1', 'Swift UVOT White']


## 3. Accessing Schedule and Observation Data

### 3.1. Schedules

In the ACROSS system, `Schedules` aggregate observation metadata for a given telescope. They are broadly grouped by status (`planned`, `scheduled`, and `as-flown`) and by fidelity (`low` and `high`). `Schedules` can be retrieved and filtered using these parameters as well as others, including date range, observatory/telescope name and ID, and external ID.

`Schedule` objects are optionally paginated--users can supply a `page` (e.g., 1) and `page_limit` (e.g. 10 per page) to limit the number of returned objects.

To retrieve all schedules matching the input parameters, use the `get_many` method:

In [15]:
# GET a paginated list of schedules:
schedules = client.schedule.get_many(page=1, page_limit=10, fidelity="low", status="planned")
print(f"Page: {schedules.page}")
print(f"Number of schedules returned: {len(schedules.items)}")
print(f"Total number of schedules: {schedules.model_dump()['total_number']}")

Page: 1
Number of schedules returned: 10
Total number of schedules: 177


Each of these returned items is a schedule for a given telescope. 

We can use the `model_dump_json` Pydantic schema method to display the schedule data in a nicer format:

In [16]:
print(schedules.items[0].model_dump_json(indent=4))

{
    "telescope_id": "281a5a5d-3629-4aa3-a739-968bee65415f",
    "name": "nustar_low_fidelity_planned_2025-11-20_2025-11-26",
    "date_range": {
        "begin": "2025-11-20T12:45:03",
        "end": "2025-11-26T14:00:00"
    },
    "status": "planned",
    "external_id": null,
    "fidelity": "low",
    "id": "7db48692-2832-4f48-9130-bac01da3bba6",
    "observations": [],
    "observation_count": 7,
    "created_on": "2025-11-22T01:07:01.071060",
    "created_by_id": "1022155f-f13d-4ade-ac73-3d1941f612da",
    "checksum": "bd82609d4f044f761b15fa5102a5bf26a92efff47b8223ef9acd1e1709aa1a7e4ee104d0c17fe32f9366ffb2b2ea4c55c98e39893363d0d4510875767f53e87f"
}


By default, getting `Schedules` will not return their associated `Observations` in order to reduce the size of the returned data. However, users can specify if they want the `Observations` belonging to that `Schedule` returned as well, using the `include_observations` flag:

In [17]:
schedule = client.schedule.get_many(
    page=1, page_limit=1, fidelity="low", status="planned", include_observations=True
)
schedule_observations = schedule.items[0].observations
print(f"Number of observations: {len(schedule_observations)}")
print(schedule_observations[0].model_dump_json(indent=4))

Number of observations: 7
{
    "instrument_id": "8e3f11f7-c943-4b45-b55e-59d475a4114f",
    "object_name": "SWIFTJ2036d0m0028",
    "pointing_position": {
        "ra": 308.98929,
        "dec": -0.4602
    },
    "date_range": {
        "begin": "2025-11-20T12:45:03",
        "end": "2025-11-21T05:50:00"
    },
    "external_observation_id": "91101647002",
    "type": "timing",
    "status": "planned",
    "pointing_angle": 0.0,
    "exposure_time": 33300.0,
    "reason": null,
    "description": null,
    "proposal_reference": null,
    "object_position": {
        "ra": null,
        "dec": null
    },
    "depth": null,
    "bandpass": {
        "anyof_schema_1_validator": null,
        "anyof_schema_2_validator": null,
        "anyof_schema_3_validator": null,
        "actual_instance": {
            "filter_name": "NuSTAR",
            "min": 0.1581431102464288,
            "max": 4.132806614440008,
            "type": "WAVELENGTH",
            "central_wavelength": 2.1454748623

If you only want to know about the most up-to-date schedules in the ACROSS system, there's also a `get_history` endpoint, which returns only the most-recently submitted schedule per telescope in a specified date-range: 

In [18]:
# GET the most recent schedules for the input parameters
recent_schedules = client.schedule.get_history(status="planned")
print(f"Number of recent schedules: {recent_schedules.model_dump()['total_number']}")

Number of recent schedules: 182


`get_history` also has an optional `include_observations` flag, which works in the same way as for `get_many`:

In [19]:
schedule = client.schedule.get_history(page=1, page_limit=1, status="planned", include_observations=True)
schedule_observations = schedule.items[0].observations
print(f"Number of observations: {len(schedule_observations)}")

Number of observations: 8497


And finally, there's a `get` endpoint to look up a schedule by its known UUID:

In [20]:
# GET a schedule by UUID
schedule = client.schedule.get(id=recent_schedules.items[0].id)
print(schedule.model_dump_json(indent=4))

{
    "telescope_id": "222d5f4a-8fd6-4299-a696-b3a6cb13d7bc",
    "name": "fermi_lat_week_913",
    "date_range": {
        "begin": "2025-11-26T23:58:55",
        "end": "2025-12-03T23:50:55"
    },
    "status": "planned",
    "external_id": null,
    "fidelity": "high",
    "id": "7a94a265-c1e0-4f53-b712-6bb5f5c539ef",
    "observations": [],
    "observation_count": 8497,
    "created_on": "2025-11-22T02:22:37.345065",
    "created_by_id": "1022155f-f13d-4ade-ac73-3d1941f612da",
    "checksum": "bfec4a59d40a0fc8e29659677229c2d49e3cfb902c00c5a0ffe75dd92e5d250ae3f5ecfd2dff6a536956792ef4dc0cd47185799a243c3495e3647085a7f549e8"
}


### 3.2. Observations

`Observation` objects contain the specific information about a telescope's exposure at a given time, such as where it was observing and with what instrument and filter. They are grouped into `Schedules` and can be accessed through the returned schedule objects described above (via `schedule.observations`), or through one of two endpoints described here.

Like `Schedule` objects, `Observations` can be optionally paginated.

To retrieve many `Observations` (across one or many `Schedules`), use the `get_many` method with any number of optional input parameters. Here we will demonstrate the most common ones:

In [21]:
# GET observations by date range
from datetime import datetime

observations = client.observation.get_many(
    page=1, page_limit=10, date_range_begin=datetime(2025, 10, 20), date_range_end=datetime(2025, 10, 21)
)
print(observations.items[0].model_dump_json(indent=4))

{
    "instrument_id": "9899d36c-9e07-4927-8295-04c4ced70f1a",
    "object_name": "NGC-1846",
    "pointing_position": {
        "ra": 76.92875,
        "dec": -67.47457
    },
    "date_range": {
        "begin": "2025-10-20T18:51:25",
        "end": "2025-10-20T22:34:25"
    },
    "external_observation_id": "9012:1:2",
    "type": "imaging",
    "status": "planned",
    "pointing_angle": 0.0,
    "exposure_time": 13380.0,
    "reason": null,
    "description": null,
    "proposal_reference": null,
    "object_position": {
        "ra": 76.92875,
        "dec": -67.47457
    },
    "depth": null,
    "bandpass": {
        "anyof_schema_1_validator": null,
        "anyof_schema_2_validator": null,
        "anyof_schema_3_validator": null,
        "actual_instance": {
            "filter_name": "F115W",
            "min": 10130.0,
            "max": 12819.999999999996,
            "type": "WAVELENGTH",
            "central_wavelength": 11474.999999999998,
            "peak_wavelength":

In [22]:
# GET observations by cone search
observations = client.observation.get_many(cone_search_ra=120.0, cone_search_dec=-50.0, cone_search_radius=1)
print(f"Number of observations returned: {observations.model_dump()['total_number']}")

Number of observations returned: 18


In [23]:
# GET observations by observatory ID
observatory = client.observatory.get_many(name="HST")[0]
observations = client.observation.get_many(observatory_ids=[observatory.id])
print(f"Number of observations returned: {observations.model_dump()['total_number']}")
print("The same process can be used to get observations by telescope, instrument, and schedule ID")

Number of observations returned: 6132
The same process can be used to get observations by telescope, instrument, and schedule ID


In [24]:
# GET observations by bandpass information
observations = client.observation.get_many(bandpass_min=2000.0, bandpass_max=3000.0, bandpass_type="angstrom")
print(f"Number of observations returned: {observations.model_dump()['total_number']}")

Number of observations returned: 4214


In [25]:
# GET observations by status
observations = client.observation.get_many(page=1, page_limit=10, status="planned")
print(f"Number of observations returned: {observations.model_dump()['total_number']}")

Number of observations returned: 111933


And as always there's a `get` method that behaves the same to the others demonstrated so far.